Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [2]:
# Вдруг пригодиться
df_copy = df.copy()

##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [3]:
df.gender.value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

In [4]:
print(f'{df.gender.value_counts().index[0]} - {df.gender.value_counts().index[1]} : {df.gender.value_counts()[0] / df.gender.value_counts()[1]}')
print(f'{df.gender.value_counts().index[1]} - {df.gender.value_counts().index[0]} : {df.gender.value_counts()[1]/ df.gender.value_counts()[0]}')

Male - Female : 1.0192087155963303
Female - Male : 0.9811533052039381


##### 2. Какое количество уникальных значений у поля InternetService?

In [5]:
f'Amount of unique valies : {len(df.InternetService.unique())}'

'Amount of unique valies : 3'

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [6]:
# Функция для удобства
def show_statistics(data):
    print(f' Median : {data.median()} \n Mean : {data.mean()} \n Std : {data.std()}')


In [7]:
# То, чтомешает жить
df.TotalCharges.value_counts().index[0]

' '

In [8]:
len(df)

7043

In [9]:
# Убираем и проверяем длинну
df_2 = df.loc[df.TotalCharges != ' ']

In [10]:
len(df_2)

7032

In [11]:
# Меняем тип данных
df_2.TotalCharges = df_2.TotalCharges.astype('float64')

/Users/Viktoryia/Documents/gb/env/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
# Профит)
show_statistics(df_2.TotalCharges)

 Median : 1397.475 
 Mean : 2283.300440841866 
 Std : 2266.771361883145


В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [13]:
df.PhoneService.value_counts()

Yes    6361
No      682
Name: PhoneService, dtype: int64

In [14]:
df.PhoneService = np.where(df.PhoneService == 'Yes', 1, 0)

In [15]:
df.PhoneService.value_counts()

1    6361
0     682
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [16]:
df.TotalCharges = df.TotalCharges.replace(' ', np.NaN).astype('float32')

In [17]:
df.TotalCharges = df.TotalCharges.fillna(0)

In [18]:
# Удаление 11 значений особо не повлияло
show_statistics(df.TotalCharges)

 Median : 1394.550048828125 
 Mean : 2279.734375 
 Std : 2266.79443359375


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [19]:
df.Churn = np.where(df.Churn == 'Yes', 1, 0)

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [20]:
columns = ['StreamingMovies', 'StreamingTV', 'TechSupport']
for el in columns:
    df[el] = np.where(df[el] == 'Yes', 1, 0)

##### 8. Заполните пропуски в поле PhoneService значением 0

In [21]:
# Нулевых значений нет
df.PhoneService.isna().sum()

0

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [22]:
# Выводим все колонки,  переводим во множество, вычитаем, удаляем
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
all_columns = list(df.columns)
diff = set(all_columns) - set(columns)
df = df.drop(diff, axis=1)
df.head(3)

,gender,tenure,PhoneService,TechSupport,StreamingTV,StreamingMovies,TotalCharges,Churn
0,Female,1,0,0,0,0,29.850000,0
1,Male,34,1,0,0,0,1889.500000,0
2,Male,2,1,0,0,0,108.150002,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [23]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                   test_size=0.25)

In [24]:
len(X_train), len(y_train)

(5282, 5282)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [26]:
gender.fit_transform(X_train)

,gender_Female,gender_Male
6607,0,1
2598,1,0
2345,1,0
4093,1,0
693,1,0
...,...,...
3772,0,1
5191,1,0
5226,0,1
5390,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [27]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [28]:
#Ваш код здесь
total_charges = Pipeline([
    ('selector', NumberSelector(key='TotalCharges')),
    ('standard', StandardScaler())
])


Объединение всех "кубиков" очень легко сделать таким образом

In [29]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [30]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', total_charges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [31]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [32]:
#Ваш код здесь
predictions = pipeline.predict_proba(X_test)

In [33]:
predictions[0]

array([0.27666667, 0.72333333])

In [34]:
preds = predictions[:, 1]

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [35]:
from sklearn.metrics import roc_auc_score, log_loss
print(f'Roc_auc_score: {roc_auc_score(y_test, preds)}')
print(f'Log_loss: {log_loss(y_test, preds)}')

Roc_auc_score: 0.8016644465361077
Log_loss: 0.6714239461701229


In [36]:
# ради любопытства
final_predict = pipeline.predict(X_test)

In [37]:
from sklearn.metrics import roc_curve, classification_report
print(classification_report(y_test, final_predict))

              precision    recall  f1-score   support

           0       0.82      0.87      0.85      1282
           1       0.59      0.50      0.54       479

    accuracy                           0.77      1761
   macro avg       0.71      0.69      0.69      1761
weighted avg       0.76      0.77      0.76      1761



### Сохраним наш пайплайн

In [38]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)